In [1]:
import pandas as pd
import numpy as np
import pickle
import rsc.recursos as rsc
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import warnings
warnings.filterwarnings('ignore')
with open("datos/libros_scrap.pkl", "rb") as alias:
    libro2 =pickle.load(alias)

df_movies = pd.read_csv("imdb_top_1000.csv")

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [ ]:
#Este loop coree el seleneum de scrapeo.
libro = []
resultado = []
for i in df_movies["Series_Title"]:
    rsc.busqueda(i,resultado)
    print(resultado)
    libro.append(resultado)
    resultado = []

In [2]:
#Corro todas mis funciones de limpieza
scrap_limpio = rsc.listas_libros(libro2)
lista_votos_limpio = [rsc.limpieza_votos(voto) for voto in scrap_limpio[2]]
nota_promedio_limpio = [rsc.promedio(rati) for rati in scrap_limpio[1]]
Autor_limpio = [rsc.Autor(auto) for auto in scrap_limpio[0]]
fuzzy_titulo = rsc.semejanza(scrap_limpio[3],list(df_movies["Series_Title"]))
df_movies["Gross"] = df_movies["Gross"].str.replace(',','')

In [3]:
#Creo las columnas en el df de peliculas para luego filtrar las peliculas que coinciden con los libros.
df_movies["Existe_libro"] = fuzzy_titulo
df_movies["Autor_libro"] = Autor_limpio
df_movies["Nota_promedio_libro"] = nota_promedio_limpio
df_movies["Votos_libro"] = lista_votos_limpio
df_movies['Nota_promedio_libro'] = df_movies['Nota_promedio_libro'].apply(lambda x: x*2)

In [4]:
#Aqui creo un DF nuevo con solo las filas que coincide que las peliculas tienen libro.
df_movies_limpio = df_movies[df_movies["Existe_libro"] == True]
#Copio las filas que me interesan.
df_libros = df_movies_limpio[["Autor_libro","Nota_promedio_libro","Votos_libro"]].copy()
#Elimino las columnas que ya no me hacen falta en el dataframe de peliculas.
df_movies_limpio.drop("Existe_libro",inplace=True,axis=1),
df_movies_limpio.drop("Nota_promedio_libro",inplace=True,axis=1)
df_movies_limpio.drop("Votos_libro",inplace=True,axis=1)
df_movies_limpio.drop("Autor_libro",inplace=True,axis=1)



In [5]:
#Creo dos nuevos csv con mis dos dataframes limpios.
df_movies_limpio.to_csv(f"datos/df_movies_limpio.csv")
df_libros.to_csv(f"datos/df_libros.csv")